In [1]:
#First step is to demultiplex each of the 5 runs. Will use AMPtk to demultiplex the data
#this will keep only amplicons where both forward and reverse primers are found, as well as discard reads < 125 bp
!for i in MHC2a MHC2b MHC3 MHC4 MHC5; \
    do amptk ion -i ../$i.fastq -m ../$i.mapping_file.txt -o $i --full_length --min_len 125; \
    done

-------------------------------------------------------
[03:24:07 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[03:24:08 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.2
[03:24:08 PM]: Foward primer: AGGGTGCTCCTCACAGCCCTGTG,  Rev comp'd rev primer: CAGGAAATKYGCYGCGGGGACA
[03:24:08 PM]: Loading FASTQ Records
[03:24:12 PM]: 2,840,048 reads (1.7 GB)
-------------------------------------------------------
[03:54:50 PM]: Concatenating Demuxed Files
[03:54:54 PM]: 2,840,048 total reads
[03:54:54 PM]: 2,517,886 valid Barcode
[03:54:54 PM]: 2,321,440 Fwd Primer found, 326,493 Rev Primer found
[03:54:54 PM]: 5,127 discarded too short (< 125 bp)
[03:54:54 PM]: 321,366 valid output reads
[03:54:55 PM]: Found 44 barcoded samples
                Sample:  Count
                 NY040:  26088
                 NY012:  25348
                 NY047:  21492
                 WI091:  15646
                 NY001:  13602
                 NY017:  13353
                 NY008:  12801
     

In [2]:
#concatenate all files together
!cat mhc2a.demux.fq mhc2b.demux.fq mhc3.demux.fq mhc4.demux.fq mhc5.demux.fq > mhc.all.demux.fq

In [3]:
!amptk show -i mhc.all.demux.fq

----------------------------------
Found 181 barcoded samples
              Sample:  Count
               NY032:  51161
               NY040:  27597
               NY012:  27376
               NY047:  25972
               NY053:  21419
               WI107:  20334
               WI091:  18855
               PA012:  17703
               WI098:  16590
               NY001:  16078
               VT006:  15094
               WI054:  14924
               NY017:  14741
               WI105:  14021
               NY008:  13976
               NY016:  13900
               PA001:  13886
               MI130:  13828
              NY033B:  13729
               WI169:  13544
               WI076:  13365
                NY34:  13348
               VT018:  13180
               NY051:  13158
               VT016:  12810
               NY004:  12751
               WI078:  12562
               WI062:  12531
               WI166:  12342
               NY014:  12063
               NY048:  11688
          

### Okay, so now all the data has been demultiplexed, we kept only full length reads because we need to anchor to the start of the DRB2-exon2 translation when running the HMM filter later on.  Obviously many of these samples will drop out due to there being too few reads.  I'm implemented the sample dropout in the MHC-DADA2 script.

In [4]:
#pull out the small subset for methods, as they likely can't be used in larger study
!amptk select -i mhc.all.demux.fq -f ../mhc.subset.methods.txt -o mhc.methods.demux.fq

Kept 74363 reads out of 1178970 total reads


In [5]:
!amptk show -i mhc.methods.demux.fq

----------------------------------
Found 17 barcoded samples
              Sample:  Count
               WI169:  13544
               WI166:  12342
               WI171:  9252
               WI168:  8124
               WI163:  5717
               IL030:  4771
               IL020:  4668
               WI165:  3531
               IL032:  3267
               IL024:  2894
               WI170:  1578
               IL022:  1553
               WI172:  1387
               WI167:  1226
               WI164:  264
               IL016:  171
               IL018:  74
----------------------------------
Read count: 74363
Read length average: 305
Read length range: 125 - 497 bp
----------------------------------


In [51]:
#now run through MHC DADA2 script, since only a few samples, don't remove alleles found only 1
#try increase quality threshold to 2.0, since we have longer reads and need the 3' end 
#to serve as an anchor, i.e. that is where translation starts
!mhc-dada2.py -i mhc.methods.demux.fq --filter_allele 1  \
              -e 2.0 --rev_comp -m ../mhc.mapping_file.txt -o methods --debug

-------------------------------------------------------
[05:30:04 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:30:04 PM]: Reverse complementing FASTQ records
[05:30:13 PM]: Loading FASTQ Records
[05:30:13 PM]: 74,363 reads (45.2 MB)
[05:30:13 PM]: Quality Filtering, expected errors < 2.0
[05:30:14 PM]: 47,638 reads passed
[05:30:14 PM]: Splitting FASTQ file by Sample
[05:30:17 PM]: Removed 6 samples with read number < 500 (--min_reads_sample)
[05:30:17 PM]: Running DADA2 pipeline
[05:31:50 PM]: R v3.3.3, DADA2 v1.3.5
[05:31:50 PM]: 306 total inferred sequences (iSeqs)
[05:31:50 PM]: 27 denovo chimeras removed
[05:31:50 PM]: 279 valid iSeqs
[05:31:50 PM]: Translating to Amino Acid Sequence in 3 frames
[05:31:50 PM]: 111 total translated sequences > 75 aa
[05:31:50 PM]: Running HMMER3 MHC HMM model
[05:31:50 PM]: 87 pass HMM filter, 84 are unique AA sequence.
[05:31:50 PM]: Mapping reads to DADA2 filtered iSeqs
[05:32:04 PM]: 48,848 reads mapped to iSeqs (66%)
[05:32

In [17]:
#now try DADA2 workflow with the rest of the data, drop the Illinois samples as small dataset and meta data is
#questionable as well as Neda mine as we don't have any qPCR data for those sample
#these samples were used for a smaller subset for methods manuscript
!amptk remove -i mhc.all.demux.fq -f ../mhc.subset.methods.txt -o mhc.large.demux.fq

Removed 17 samples
Kept 1104607 reads out of 1178970 total reads


In [53]:
#now run through DADA2
!mhc-dada2.py -i mhc.large.demux.fq -e 2.0 --rev_comp -m ../mhc.mapping_file.txt -o all 

-------------------------------------------------------
[05:44:00 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:44:00 PM]: Reverse complementing FASTQ records
[05:46:03 PM]: Loading FASTQ Records
[05:46:04 PM]: 1,104,607 reads (639.3 MB)
[05:46:04 PM]: Quality Filtering, expected errors < 2.0
[05:46:18 PM]: 861,929 reads passed
[05:46:18 PM]: Splitting FASTQ file by Sample
[05:47:22 PM]: Removed 27 samples with read number < 500 (--min_reads_sample)
[05:47:22 PM]: Running DADA2 pipeline
[09:13:26 PM]: R v3.3.3, DADA2 v1.3.5
[09:13:26 PM]: 1,758 total inferred sequences (iSeqs)
[09:13:26 PM]: 120 denovo chimeras removed
[09:13:26 PM]: 1,638 valid iSeqs
[09:13:26 PM]: Translating to Amino Acid Sequence in 3 frames
[09:13:26 PM]: 657 total translated sequences > 75 aa
[09:13:26 PM]: Running HMMER3 MHC HMM model
[09:13:27 PM]: 483 pass HMM filter, 436 are unique AA sequence.
[09:13:27 PM]: Mapping reads to DADA2 filtered iSeqs
[09:17:05 PM]: 917,765 reads mapped to iSeq

### This is somewhat discouraging, that when I try to look at the "methods" part of the data, that it doesn't display the same amount of allelic diversity as the larger datset.  So to check if any of these data have structure, I'll run the data through a hyopthesis test for each of the meta data variables.  For this I will use an R script that runs iteratively through vegan adonis test followed by ordination in Phyloseq to visualize the data.

In [20]:
#This script loops through all of the treatments or meta data variables, it removes any samples that contain "no_data"
#and then takes that data and does several tests. Hypothesis tests (adonis) and dispersion (betadisper() followed by
#drawing the data in an NMDS ordination.  It does this with 4 distance metrics (RaupCrick, BrayCurtis, Unifrac, and
#Weighted Unifrac).  The Unifrac method relies on a protein level phylogenic tree of the MHC alleles. Finally, the script
#calculates alpha-diversity for each of the treatments.
!$HOME/amptk/util/phyloseq_graphs.R all.update.biom all.tree.phy all

Loading required package: phyloseq
Loading required package: methods
Loading required package: ggplot2
Loading required package: plyr
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.4-3
Loading required package: RColorBrewer
Loading required package: gridExtra
Loading required package: grid
[1] ‘1.19.1’
[1] ‘2.2.1’
[1] ‘1.8.4’
[1] ‘2.4.3’
[1] ‘1.1.2’
[1] ‘2.2.1’
[1] ‘3.3.1’
 [1] "CT_result"            "phinchID"             "CT_mean"             
 [4] "age_class"            "DNA_quant_result"     "BarcodeSequence"     
 [7] "site"                 "sex"                  "site_date"           
[10] "LinkerPrimerSequence" "WNS_status"           "date"                
[13] "DNA_quant_mean"       "ReversePrimer"        "location"            
[1] "CT_result"
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 456 taxa and 128 samples ]
sample_data() Sample Data:       [ 128 samples by 15 sample variable

![SiteMap](no_IL_Neda/test.location.ordinations.pdf)

#### There seems to be very little to no structure in these data, even between Eastcoast (NY,VT,PA) vs Midwest (WI,MI).  
##### I ran adonis test in R for each variable and then betadispersion


In [19]:
#This script grabs the raw data and formats into a table.
!$HOME/amptk/util/parse_adonis.py no_IL_Neda

                 braycurtis            raupcrick            unifrac            weighted_unifrac           
                     Adonis Betadisper    Adonis Betadisper  Adonis Betadisper           Adonis Betadisper
CT_result            0.8291      0.462    0.5984      0.566  0.2384      0.979           0.7584      0.985
DNA_quant_mean       0.0226      0.001    0.3169      0.001  0.4694      0.001           0.2096      0.001
CT_mean              0.9017      0.001    0.7921      0.045  0.5455      0.001           0.4727      0.001
age_class            0.0669      0.007    0.0024      0.189  0.0029      0.409           0.3087      0.428
site_date            0.5249      0.063    0.2100      0.005  0.1837      0.313           0.2051      0.307
site                 0.5069      0.035    0.1437      0.007  0.1260      0.196           0.1733      0.180
sex                  0.6164      0.978    0.5453      0.556  0.1387      0.068           0.8431      0.071
DNA_quant_result     0.7763      0.28

![Alpha diversity by STATUS](no_IL_Neda/test.WNS_status.alpha_diversity.pdf)

#### Of course Michelle you can look at these data in a lot more detail than I am able to, but if there are not even site differences or geographic differences between populations.  So I think we need another genetic marker to show what is happening here, i.e. can we differientiate between groups of LBB by geographic location or not?  Are there simply too many MHC alleles?  Or is the answer something along the lines that there are huge numbers of MHC alleles, however does not seem to correlate with WNS?  Perhaps we sampled much too early, i.e. lifespan is ~20 years, may take generations before you see any bottleneck or stabilization?

#### So I went back and put all the data together, i.e. the entire MHC dataset

In [14]:
#now run through DADA2
!mhc-dada2.py -i mhc.all.demux.fq -e 2.0 --rev_comp -m ../mhc.mapping_file.txt -o mhc 

-------------------------------------------------------
[09:55:37 AM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[09:55:37 AM]: Reverse complementing FASTQ records
[09:57:41 AM]: Loading FASTQ Records
[09:57:41 AM]: 1,178,970 reads (684.5 MB)
[09:57:41 AM]: Quality Filtering, expected errors < 2.0
[09:57:56 AM]: 909,567 reads passed
[09:57:56 AM]: Splitting FASTQ file by Sample
[09:59:00 AM]: Removed 33 samples with read number < 500 (--min_reads_sample)
[09:59:00 AM]: Running DADA2 pipeline
[07:21:44 PM]: R v3.3.1, DADA2 v1.5.0
[07:21:44 PM]: 1,883 total inferred sequences (iSeqs)
[07:21:44 PM]: 129 denovo chimeras removed
[07:21:44 PM]: 1,754 valid iSeqs
[07:21:44 PM]: Translating to Amino Acid Sequence in 3 frames
[07:21:45 PM]: 694 total translated sequences > 75 aa
[07:21:45 PM]: Running HMMER3 MHC HMM model
[07:21:45 PM]: 507 pass HMM filter, 459 are unique AA sequence.
[07:21:56 PM]: BIOM OTU table created: mhc.biom
[07:21:56 PM]: Generating phylogenetic tree
----

In [16]:
#run phyloseq/vegan script
!$HOME/amptk/util/phyloseq_graphs.R mhc.biom mhc.tree.phy mhc

Loading required package: phyloseq
Loading required package: methods
Loading required package: ggplot2
Loading required package: plyr
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.4-3
Loading required package: RColorBrewer
Loading required package: gridExtra
Loading required package: grid
[1] ‘1.19.1’
[1] ‘2.2.1’
[1] ‘1.8.4’
[1] ‘2.4.3’
[1] ‘1.1.2’
[1] ‘2.2.1’
[1] ‘3.3.1’
 [1] "CT_result"            "phinchID"             "CT_mean"             
 [4] "age_class"            "DNA_quant_result"     "BarcodeSequence"     
 [7] "site"                 "sex"                  "site_date"           
[10] "LinkerPrimerSequence" "WNS_status"           "date"                
[13] "DNA_quant_mean"       "ReversePrimer"        "location"            
[1] "CT_result"
phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 486 taxa and 126 samples ]
sample_data() Sample Data:       [ 126 samples by 15 sample variable

In [17]:
#summarize the statistics 
!$HOME/amptk/util/parse_adonis.py phyloseq_all

                 braycurtis            raupcrick            unifrac            weighted_unifrac           
                     Adonis Betadisper    Adonis Betadisper  Adonis Betadisper           Adonis Betadisper
CT_result            0.4170     0.2756    0.3010     0.8094  0.7191     0.5817           0.2748     0.5675
DNA_quant_mean       0.0169     0.0001    0.2900     0.0001  0.8116     0.0001           0.0366     0.0001
CT_mean              0.9622     0.0001    0.9296     0.0039  0.3744     0.0002           0.1261     0.0002
age_class            0.0439     0.0009    0.0048     0.1028  0.0186     0.5920           0.0102     0.5967
site_date            0.4262     0.0575    0.0704     0.1199  0.2292     0.0150           0.3872     0.0138
site                 0.2501     0.0284    0.0337     0.1437  0.1629     0.0352           0.3001     0.0352
sex                  0.6454     0.9867    0.2781     0.1570  0.9151     0.0117           0.4475     0.0104
DNA_quant_result     0.3542     0.211

### We still have very weak structure here...
![SiteMap](mhc_phyloseq_all/mhc.location.ordinations.pdf)

![SiteMap](mhc_phyloseq_all/mhc.location.alpha_diversity.pdf)